**Links:**
* Wikipedia: https://en.wikipedia.org/wiki/Boggle
* How to play? https://www.wikihow.com/Play-Boggle
* English dice configuration: https://boardgames.stackexchange.com/questions/29264/boggle-what-is-the-dice-configuration-for-boggle-in-various-languages
* More configurations: https://en.wikipedia.org/wiki/Talk%3aBoggle#Boggle_Variants
* Inspiration for solver [1]: https://medium.com/@ashalabi13/solving-boggle-using-depth-first-searches-and-prefix-trees-9c3faa89ea99
* [2] https://www.geeksforgeeks.org/boggle-find-possible-words-board-characters/

In [1]:
import numpy as np

In [2]:
# Create a random board with given dice
def create_board(dice):
    board = [list(zeros) for zeros in np.zeros((4,4))]
    for i in range(4):
        for j in range(4):
            # Take a random die from the set of dice
            die = dice.pop(np.random.randint(len(dice)))
            # Assign a random letter from the die onto the board
            board[i][j] = die[np.random.randint(6)]
    return board

In [3]:
# Get adjacent vertices for given vertex
def get_adj(i,j):
    adj_indices = [(i-1,j), (i+1,j), (i,j-1), (i,j+1), (i-1,j-1), (i+1,j-1), (i-1,j+1), (i+1,j+1)]
    return list(filter(lambda x : x[0] in range(0,4) and x[1] in range(0,4), adj_indices))

# For quick access save adjacent vertices into a map
adj_map = dict()
for i in range(4):
    for j in range(4):
        adj_map[(i,j)] = get_adj(i,j)

In [4]:
# Depth-First Search from lecture slides
def DFS(vertices, edges, prefix_tree, dictionary):
    un_visited_vertices = [[False for j in range(4)] for i in range(4)]
    #un_visited_vertices = [(i,j) for i in range(4) for j in range(4)]
    for i in range(4):
        for j in range(4):
            string = ""#vertices[i][j]
            visited_vertices = [i[:] for i in un_visited_vertices]
            #print("starting from", vertices[i][j], "at", (i,j))
            DFS_visit((i,j), vertices, edges, visited_vertices, string, prefix_tree, dictionary)
            
def DFS_visit(vertice, vertices, edges, visited_vertices, string, prefix_tree, dictionary):
    #global comparisons
    #global all_comparisons
    string += vertices[vertice[0]][vertice[1]]
    #print(string.lower())
    visited_vertices[vertice[0]][vertice[1]] = True
    if len(string) == 3:
        if prefix_tree.get(string.lower()) == None:
            return
    if dictionary.get(string.lower()) != None:
        print(string)
    # Find adjacent vertices
    adj = edges[vertice]
    for adj_v in adj:
        if visited_vertices[adj_v[0]][adj_v[1]] == False:
            branch_visited_vertices = [i[:] for i in visited_vertices]
            DFS_visit(adj_v, vertices, edges, branch_visited_vertices, string, prefix_tree, dictionary)
    # Backtrack
    string = string.rstrip(string[-1])
    visited_vertices[vertice[0]][vertice[1]] = False

In [5]:
# en: https://boardgames.stackexchange.com/questions/29264/boggle-what-is-the-dice-configuration-for-boggle-in-various-languages
dice = [["R", "I", "F", "O", "B", "X"],
       ["I", "F", "E", "H", "E", "Y"],
       ["D", "E", "N", "O", "W", "S"],
       ["U", "T", "O", "K", "N", "D"],
       ["H", "M", "S", "R", "A", "O"],
       ["L", "U", "P", "E", "T", "S"],
       ["A", "C", "I", "T", "O", "A"],
       ["Y", "L", "G", "K", "U", "E"],
       ["Qu", "B", "M", "J", "O", "A"],
       ["E", "H", "I", "S", "P", "N"],
       ["V", "E", "T", "I", "G", "N"],
       ["B", "A", "L", "I", "Y", "T"],
       ["E", "Z", "A", "V", "N", "D"],
       ["R", "A", "L", "E", "S", "C"],
       ["U", "W", "I", "L", "R", "G"],
       ["P", "A", "C", "E", "M", "D"],]

board = create_board(dice)

In [6]:
### TEST
dictionary = {"test":"", "boggle":"", "his":"", "josh":"", "toe":"", "joe":"", "she":""}
prefix_tree = {"tes": "test", "bog": "boggle", "his": "his", "jos": "josh", "toe": "toe", "joe": "joe", "she":"she"}
board = [['T', 'E', 'S', 'T'],
         ['B', 'O', 'H', 'L'],
         ['G', 'J', 'I', 'H'],
         ['G', 'L', 'E', 'S']]
found_words = DFS(board, adj_map, prefix_tree, dictionary)
print(found_words)
###

TEST
TOE
SHE
BOGGLE
HIS
JOE
JOSH
HIS
SHE
None


In [7]:
board

[['T', 'E', 'S', 'T'],
 ['B', 'O', 'H', 'L'],
 ['G', 'J', 'I', 'H'],
 ['G', 'L', 'E', 'S']]